In [1]:
using JuMP, Plots, HiGHS, Distributions, Statistics, PlotlyJS
plotlyjs()
# Define the second stage problem and f(x) to make it easier to evaluate the cost of buying x newspapers. 
function Second_Stage(x,d)
    m = Model(HiGHS.Optimizer)
    set_silent(m)
    @variables(m, 
    begin
        y ≥ 0
        w ≥ 0
    end)

    @constraints(m,
    begin
        y ≤ d
        y + w ≤ x
    end)

    @objective(m, Min, -25 * y - 5 * w)
    optimize!(m)
    return objective_value(m)
end

f(x, d) = 10*x + Second_Stage(x,d)

#SAA for the newsvendor problem

# M is the number of replicas of the demand distribution
M = 10
# possible amounts of samples to be drawn from the distribution for the lower bound
N_amounts = collect(50:50:1000)
cont_unif = Uniform(50, 150)



function Recourse_function(d)
    m = Model(HiGHS.Optimizer)
    set_silent(m)
    n = length(d)
    @variables(m,
    begin
        0 ≤ x ≤ 150
        y[1:n] ≥ 0
        w[1:n] ≥ 0
    end)
    
    @constraints(m,
    begin
        ct1[s=1:n], y[s] ≤ d[s]
        ct2[s=1:n], y[s] + w[s] ≤ x
    end)
    
    @objective(m, Min, 10 * x + (1/n)*sum((-25*y[s] -5*w[s]) for s = 1:n))
    
    optimize!(m)
    
    return objective_value(m), value(x)
end

WebIO._IJuliaInit()

Recourse_function (generic function with 1 method)

In [2]:
LB_V = zeros(length(N_amounts))
std_V = zeros(length(N_amounts))
x_matrix = zeros(length(N_amounts), M)

for (index, N) in enumerate(N_amounts)
    demand_matrix = rand(cont_unif, M, N)
    lower_bound = 0
    lower_bound_samples = zeros(M)
    x_samples = zeros(M)
    for i in 1:M
        lower_bound_samples[i], x_samples[i] = Recourse_function(demand_matrix[i,:])
        lower_bound += lower_bound_samples[i]
    end
    x_matrix[index, :] = x_samples
    std_V[index] = std(lower_bound_samples)
    LB_V[index] = lower_bound/M
end

In [3]:
display(LB_V)

20-element Vector{Float64}:
 -1346.227775698615
 -1308.5840944905349
 -1320.1944037194853
 -1340.1374888511473
 -1292.8631625991907
 -1325.8371786571643
 -1322.1588042636508
 -1311.9942752667932
 -1323.7187110926036
 -1304.9924091513851
 -1316.3818295904907
 -1302.6498291453622
 -1309.6975502628425
 -1304.854561261868
 -1323.9682472959337
 -1309.4016142727392
 -1296.8563045543578
 -1311.799033267543
 -1310.3584626064094
 -1315.2681400779215

In [4]:
y = collect(1:1:20)
quantiles = zeros(20, 2)
normal = Normal(0, 1)
quantil = quantile(normal, 0.95)

for i in 1:length(N_amounts)
    quantiles[i,1] = LB_V[i] - (quantil*std_V[i]/sqrt(N_amounts[i]))
    quantiles[i,2] = LB_V[i] + (quantil*std_V[i]/sqrt(N_amounts[i]))
end

labels = permutedims(["N = $i" for i in N_amounts])

Plots.plot([y;;y]', quantiles', palette = :darktest, linewidth = 2, legend = :topright, xlabel = "X", ylabel = "Number of samples divided by 50", title = "Confidence intervals of the LB", size = (800, 600), label = labels)

In [5]:
## Computing upper bound

# Use the 10 candidate solutions in 1000 out of sample scenarios to compute the upper bound (x_matrix[20,:])
K = 1000
out_of_sample = rand(cont_unif, M, K)
UB_V = zeros(M)
UB_std_V = zeros(M)

for (i,x) in enumerate(x_matrix[20,:])
    results = [Second_Stage(x, out_of_sample[i,j]) for j in 1:K]
    UB_std_V[i] = std(10*x .+ results)
    UB_V[i] = 10*x + mean(results)
end

In [6]:
display(UB_V)
display(UB_std_V)

10-element Vector{Float64}:
 -1333.7894962322262
 -1322.5336148163656
 -1345.4911903308553
 -1307.936444628409
 -1337.3563125765188
 -1284.8403019148043
 -1324.788263244864
 -1293.7491452192712
 -1302.9663941275123
 -1313.7265105590775

10-element Vector{Float64}:
 509.2418873826822
 485.78427218862197
 499.8958761280746
 489.3789000188148
 500.0668073761257
 499.6959419333998
 486.2488844539414
 481.81290362286484
 508.1382434019521
 492.34392048328044

In [7]:
y = collect(1:1:10)
quantiles = zeros(10, 2)
normal = Normal(0, 1)
quantil = quantile(normal, 0.95)

for i in 1:length(x_matrix[20, :])
    quantiles[i,1] = UB_V[i] - (quantil*UB_std_V[i]/sqrt(K))
    quantiles[i,2] = UB_V[i] + (quantil*UB_std_V[i]/sqrt(K))
end

labels = permutedims(["x_$i" for i in 1:10])

Plots.plot([y;;y]', quantiles', palette = :darktest, title="Confidence intervals of the UB", linewidth = 2, legend = :topright, label=labels)

In [28]:
## The chosen x was x_6, because it's the most conservative upper bound available, given that it encompasses a range 
## of lower profits.

## obtaining x_6
chosen_x̂ = x_matrix[20,6]

## same procedure for the UB above, but now we vary the amount of out of sample scenarios from 100 to 1000
K_vector = collect(100:100:1000)
new_UB_V = zeros(length(K_vector))
new_UB_std = zeros(length(K_vector))
for (index, K) in enumerate(K_vector)
    out_of_sample = rand(cont_unif, K)
    results = [Second_Stage(chosen_x̂, out_of_sample[j]) for j in 1:K]
    new_UB_std[index] = std(10*chosen_x̂ .+ results)
    new_UB_V[index] = 10*chosen_x̂ + mean(results)
end

In [29]:
display(new_UB_V)
display(new_UB_std)

10-element Vector{Float64}:
 -1354.6377697352134
 -1309.278893979758
 -1328.9354722568285
 -1361.4931607221354
 -1310.3173413896209
 -1283.3373355044935
 -1286.7422329698522
 -1345.45406674408
 -1316.6489840188422
 -1327.0884254062478

10-element Vector{Float64}:
 518.7172895097292
 501.6251139578795
 492.35026520316245
 514.9603810093345
 491.81071700890806
 518.9925139730273
 518.7138643251412
 508.18822151875713
 498.4006340055346
 508.27684507758437

In [30]:
y = collect(1:1:10)
quantiles = zeros(10, 2)
normal = Normal(0, 1)
quantil = quantile(normal, 0.95)

for i in 1:length(K_vector)
    quantiles[i,1] = new_UB_V[i] - (quantil*new_UB_std[i]/sqrt(K_vector[i]))
    quantiles[i,2] = new_UB_V[i] + (quantil*new_UB_std[i]/sqrt(K_vector[i]))
end

labels = permutedims(["K = $i" for i in K_vector])

Plots.plot([y;;y]', quantiles', palette = :darktest, title="Confidence interval for differing K", linewidth = 2, legend = :topright, label=labels)

In [31]:
##Compute probabilistic GAP

N = 1000
M = 10
gaps = zeros(10)
gaps_std = zeros(10)
demand_matrix = rand(cont_unif, M, N)

for (index, x) in enumerate(x_matrix[20,:])
    overcost_vector = zeros(M)
    for i in 1:M
        overcost_vector[i] = (10*x + sum(Second_Stage.(x, demand_matrix[i, :]))/N) - Recourse_function(demand_matrix[i, :])[1]
    end
    gaps[index] = mean(overcost_vector)
    gaps_std[index] = std(overcost_vector)
end

In [32]:
display(gaps)
display(gaps_std)

10-element Vector{Float64}:
 1.0232462820144748
 0.4220552028535394
 0.3906715864746957
 0.25110374418670744
 0.30361773264660313
 0.74623488202717
 0.33738495930595036
 0.33180318907132006
 0.4151777708377494
 0.27499110111782554

10-element Vector{Float64}:
 0.9914356443566213
 0.5465396968359665
 0.507530058234799
 0.3229685440103481
 0.394391278638744
 0.8208346568075847
 0.4450701752057103
 0.437985134343044
 0.5365669516740081
 0.36155679085286013

In [33]:
std_gap = std(gaps)
quantiles = zeros(10, 2)

quantiles[:,1] = gaps .- quantil.*std_gap./sqrt(1000)
quantiles[:,2] = gaps .+ quantil.*std_gap./sqrt(1000)

labels = permutedims(["x_$i" for i in 1:10])

Plots.plot([y;;y]', quantiles', palette = :darktest, title="Confidence interval of gap for all candidate solutions", linewidth = 2, legend = :topright, label=labels)


In [34]:
# And here i would choose x_4, given that the confidence interval for its gap is the smallest of all the other candidate
# solutions